# DATA EXPLORATION: INTERACTIVE DASHBOARDS

In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(URL, header = 0)
print(spacex_df.head())
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
print(max_payload, min_payload)

   Unnamed: 0  Flight Number  Launch Site  class  Payload Mass (kg)  \
0           0              1  CCAFS LC-40      0                0.0   
1           1              2  CCAFS LC-40      0                0.0   
2           2              3  CCAFS LC-40      0              525.0   
3           3              4  CCAFS LC-40      0              500.0   
4           4              5  CCAFS LC-40      0              677.0   

  Booster Version Booster Version Category  
0  F9 v1.0  B0003                     v1.0  
1  F9 v1.0  B0004                     v1.0  
2  F9 v1.0  B0005                     v1.0  
3  F9 v1.0  B0006                     v1.0  
4  F9 v1.0  B0007                     v1.0  
9600.0 0.0


#### Create an interactive dashboard.

In [4]:
# Create a dash application
app = dash.Dash(__name__)

options = ['ALL']
options.extend(spacex_df['Launch Site'].unique())
options = [{'label':'All Sites' if v == 'ALL' else v, 'value':v} for v in options]

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # Dropdown: launch Site selection.
                                dcc.Dropdown(id='site-dropdown', placeholder='Select a Launch Site here',searchable=True,options=options),
                                html.Br(),

                                # Graph 1: Success rate by site. 
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                # Slider: payload mass. 
                                html.P("Payload range (Kg):"),
                                dcc.RangeSlider(id='payload-slider', min=min_payload, max=max_payload, step=int((max_payload-min_payload)/50), value=[min_payload, max_payload]),

                                # Graph 2: Scatterplot of success rate by payload mass. 
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# Callback function 1. 
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', names='Launch Site', title='Success by Launch Site')
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]['class'].value_counts().to_frame().reset_index()
        #filtered_df['result'] = ['success' if v==1 else 'falied' for v in filtered_df['class'].values]
        fig = px.pie(filtered_df, values='count', names='class', title='Success count'+ (' for '+entered_site) if entered_site else '')
    return fig

# Callback function 2. 
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id="payload-slider", component_property="value")])

def get_scatter_chart(entered_site, payload):

    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)']>=payload[0]) & (spacex_df['Payload Mass (kg)']<=payload[1])]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', title='Success by Payload Mass',color='Booster Version Category')
    else:
        # return the outcomes piechart for a selected site
        filtered_df = filtered_df[filtered_df['Launch Site']==entered_site]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', title='Success by Payload Mass for',color='Booster Version Category')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(port = 8051)